In [ ]:
docket_original = list(new_df['Original Docket Text'])

In [ ]:
output = []
for i in range(5):
    org_str = []
    name_str = []
    stripped_str1 = []
    stripped_str2 = []

    tokens = word_tokenize(docket_original[i])
    for SFlabel, NLlabel, token in zip(tagger.tag(tokens), tree2conlltags(ne_chunk(pos_tag(tokens))), tokens):
        print(SFlabel, NLlabel, token)
        if SFlabel[1] == 'ORGANIZATION':
            org_str.append(SFlabel[0])
            stripped_str1.append('-ORG-')
        elif SFlabel[1] == 'PERSON':
            name_str.append(SFlabel[0])
            stripped_str1.append('-NAME-')
        else:
            stripped_str1.append(token)
            stripped_str2.append(token)

    output.append([docket_original[i],
                   ' '.join(org_str),
                   ' '.join(name_str),
                   ' '.join(stripped_str1),
                   ' '.join(stripped_str2)])
    
for i in range(5):
    print('docket text:', i)
    print(output[i], '\n')

In [ ]:
filename = 'docket_texts/train/DT/basic_df.pickle'
#to load
with open(filename, 'rb') as handle:
    NER_df = pickle.load(handle)
    
new_df = NER_df.copy()
print(new_df.shape)
new_df.head()

In [ ]:
### Normalize
def normalize(docket_original):

    url_regex1 = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    url_regex2 = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    date_regex = '(\d{1,2}[\/ ](\d{2}|January|Jan|February|Feb|March|Mar|April|Apr|May|May|June|Jun|July|Jul|August|Aug|September|Sep|October|Oct|November|Nov|December|Dec)[\/ ]\d{2,4})'
    punct_regex = r"[^a-zA-Z0-9]"
    num_regex = "\d+"
    extraspace_regex = " +"
    docket_original = docket_original.split(' ')
    docket_normalized = [text.lower() for text in docket_original]
    docket_nourl1 = [re.sub(url_regex1, "url", text) for text in docket_normalized]
    docket_nourl2 = [re.sub(url_regex2, "url", text) for text in docket_nourl1]
    docket_nodate = [re.sub(date_regex, "date", text) for text in docket_nourl2]
    docket_nopunct = [re.sub(punct_regex, " ", text) for text in docket_nodate]
    docket_nonum = [re.sub(num_regex, " ", text) for text in docket_nopunct]
    docket_noextraspace = re.sub(extraspace_regex, " ", ' '.join(docket_nonum))
    
    return docket_noextraspace

In [ ]:
t0 = time.time()
new_df['normalized'] = new_df['Stripped Org and Name'].apply(lambda x: normalize(x))
print('took {} seconds'.format(time.time() - t0))

In [ ]:
def tokenize(docket_noextraspace):
    return word_tokenize(docket_noextraspace)

In [ ]:
t0 = time.time()
new_df['tokenize'] = new_df['normalized'].apply(tokenize)
print('took {} seconds'.format(time.time() - t0))

In [ ]:
def stopwords(docket_tokenized):  
    return [word for word in docket_tokenized if word not in sw]

In [ ]:
t0 = time.time()
new_df['nostop'] = new_df['tokenize'].apply(stopwords)
print('took {} seconds'.format(time.time() - t0))

In [ ]:
#this is updated, comparing mannual_topics_20180828 and manual_topics_20180911 and manual_topics_20180919
manual_topics_df = pd.read_csv('manual_topics_20180919.csv') 
manual_topics_df = manual_topics_df.apply(lambda x: x.astype(str).str.lower())
manual_topics_dict = manual_topics_df.to_dict('list')
for topic in manual_topics_dict.keys():
    manual_topics_dict[topic] = [keyword for keyword in manual_topics_dict[topic] if keyword != 'nan']

In [ ]:
#this is updated, comparing mannual_topics_20180828 and manual_topics_20180911 and manual_topics_20180919
manual_topics_df = pd.read_csv('manual_topics_20180919.csv') 
manual_topics_df = manual_topics_df.apply(lambda x: x.astype(str).str.lower())
manual_topics_dict = manual_topics_df.to_dict('list')
for topic in manual_topics_dict.keys():
    manual_topics_dict[topic] = [keyword for keyword in manual_topics_dict[topic] if keyword != 'nan']

In [ ]:
#output a list of topics
def mannual_topic_assignment(text):
    #text = text.split()
    #print(text)
    output = []
    for topic in manual_topics_dict.keys():
        for keyword in manual_topics_dict[topic]:
            if ' '.join(text).find(keyword) != -1:
                output.append(topic)
    #print(output)
    #if there's motion to dismiss, then remove noaction
    if ' '.join(text).find('motion dismiss') != -1 and 'NoAction' in output:
        output = list(set(output))
        output.remove('NoAction')
    return ', '.join(set(output))

In [ ]:
docket_texts_output = list(new_df['nostop'])

docket_texts_output_DT = []
topics_DT = []
NoAction = []

for text in docket_texts_output:
    topic = mannual_topic_assignment(text)
    #print(text, topic)
    if topic != '':
        docket_texts_output_DT.append('')
        topics_DT.append(topic)
        if 'NoAction' in topic:
            NoAction.append(1)
        else:
            NoAction.append(0)
    else:
        docket_texts_output_DT.append(text)
        topics_DT.append('')
        NoAction.append(0)

In [ ]:
new_df['DT Topics'] = pd.Series(topics_DT)
new_df['NoAction'] = pd.Series(NoAction)
new_df['Removed unnecessary POS & vocab DT'] = pd.Series(docket_texts_output_DT)
print(new_df.shape)
new_df.head()

In [ ]:
def lemm(docket_nostop):
    return ' '.join([WordNetLemmatizer().lemmatize(word, pos='v') for word in docket_nostop])

In [ ]:
t0 = time.time()
new_df['lemmed'] = new_df['Removed unnecessary POS & vocab DT'].apply(lemm)
print('took {} seconds'.format(time.time() - t0))
new_df.shape

In [ ]:
docket_phrase2 = list(new_df['lemmed'])
docket_phrase2[:5]

In [ ]:
unigram_sentences_filepath = 'docket_texts/train/DT/unigram_nltk_newsop2.txt'

In [ ]:
%%time
# turn the lemmatized corpus into unigram sentences
with codecs.open(unigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for sentence in docket_phrase2:
        f.write(sentence + '\n')

In [ ]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [ ]:
bigram_model_filepath = 'docket_texts/train/DT/bigram_model_newsop2' 

In [ ]:
%%time

# store our bigram model
bigram_model = Phrases(unigram_sentences)
bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk if we don't want to run this again
#bigram_model = Phrases.load(bigram_model_filepath)

In [ ]:
bigram_sentences_filepath = 'docket_texts/train/DT/bigram_sentences_newsop2.txt'

In [ ]:
%%time

# apply the bigram model, and write it to file
with codecs.open(bigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = ' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

In [ ]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [ ]:
print('\nUnigram sentence:')
for unigram_sentence in it.islice(unigram_sentences, 0, 10):
    print(' '.join(unigram_sentence))
print('\nBigram sentence:')
for bigram_sentence in it.islice(bigram_sentences, 0, 10):
    print(' '.join(bigram_sentence))

In [ ]:
trigram_model_filepath = 'docket_texts/train/DT/trigram_model_newsop2'

In [ ]:
trigram_sentences_filepath = 'docket_texts/train/DT/trigram_sentences_newsop2.txt'

In [ ]:
%%time

with codecs.open(trigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for bigram_sentence in bigram_sentences:
        #print('Bi', bigram_sentence)
        trigram_sentence = ' '.join(trigram_model[bigram_sentence])
        #print('Tri', trigram_sentence)
        f.write(trigram_sentence + '\n')

In [ ]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [ ]:
start = 0
finish = 5
print('Original text:')
print(docket_phrase2[start:finish], '\n')

print('\nUNIGRAM Sentence:')
for unigram_sentence in it.islice(unigram_sentences, start, finish):
    print(' '.join(unigram_sentence))
print('\nBIGRAM Sentence:')
for bigram_sentence in it.islice(bigram_sentences, start, finish):
    print(' '.join(bigram_sentence))
print('\nTRIGRAM Sentence:')
for trigram_sentence in it.islice(trigram_sentences, start, finish):
    print(' '.join(trigram_sentence))

In [ ]:
def trigram_transform(texts):
    display = False
    texts = str(texts)
    trigram_output = ''
    #print(texts)

    remove_trigram = ['calendar_day', 'court_notice_intend', 'minute_entry_proceeding_hold', 'court_reportertranscriber_abovecaptioned_matter',
                      'redaction_calendar_day', 'rule_statement', 'obtain_pacer', 'may_obtain_pacer', 'reportertranscriber_abovecaptioned_matter',
                      'redact_transcript_deadline', 'send_chamber', "official_transcript_notice_give", "notice_intent_request", "proceed_hold", 
                      "fee_receipt_number", "civil_procedure", "pursuant_frcp", "official_transcript_conference", 
                      "purchase_reportertranscriber_deadline_release", "et_al", "mail_chamber", "transcript_restriction", "redaction_transcript", 
                      "transcript_view_public_terminal", "transcript_make_remotely", "associated_et_al", "electronically_available_public_without", 
                      "genesys_id", "release_transcript_restriction", "adar_bay", "redaction_request_due", "new_york", "official_transcript_conference", 
                      "transcript_make_remotely", "transcript_proceeding_conference_hold", "redaction_transcript",
                      'affidavit_jr._c.p.a', 'corporate_parent', 'certain_underwriter', 'federal_rule_civil_procedure', 'redaction_request', 
                      'official_transcript', 'rule_disclosure', 'rule_corporate_disclosure', 'place_vault', 'public_without_redaction_calendar', 
                      'purchase_deadline_release_transcript', 'transcript_proceeding_hold', 'transcript_remotely_electronically_available',
                      'minute_entry_hold', 'discovery_hear_hold', 'jury_trial_hold', "sign_judge",'place_vault']

    if texts == None:
        return None
    
    unigram_review = []
    for word in texts.split():
        unigram_review.append(word)
    if display:
        print('Uni: ', unigram_review)
    bigram_review = bigram_model[unigram_review]
    if display:
        print('Bi: ', bigram_review)
    trigram_review = trigram_model[bigram_review]
    if display:
        print('Tri: ', trigram_review)
    trigram_review = [phrase for phrase in trigram_review if phrase not in remove_trigram]
    if display:
        print('Tri removed: ', trigram_review)
    trigram_output += ' '.join(trigram_review)
    
    return trigram_output

In [ ]:
docket_phrase3 = [trigram_transform(text) for text in docket_phrase2]

In [ ]:
print(docket_phrase2[:5])
print(docket_phrase3[:5])
len(set(docket_phrase3))

In [ ]:
new_df['phrases'] = pd.Series(docket_phrase3)
new_df.shape

In [ ]:
new_df.columns

In [ ]:
i = 1
print('orignal text: ')
print(new_df['Original Docket Text'].iloc[i], '\n')
print('org and name removed: ')
print(new_df['Stripped Org and Name'].iloc[i], '\n')
print('normalized: ')
print(new_df['normalized'].iloc[i], '\n')
print('stopwords removed: ')
print(' '.join(new_df['nostop'].iloc[i]), '\n')
print('after lemmetization: ')
print(new_df['lemmed'].iloc[i], '\n')
print('after phrase modeling: ')
print(new_df['phrases'].iloc[i], '\n')

In [ ]:
remove_pos = list(pd.read_excel('NLP_to_be_removed.xlsx', sheetname = 0, header = None)[0])
remove_word = list(pd.read_excel('NLP_to_be_removed.xlsx', sheetname = 1, header = None)[0])
remove_trigram = list(pd.read_excel('NLP_to_be_removed.xlsx', sheetname = 2, header = None)[0])

In [ ]:
def remove_unnecessary(text):
    new_text = []
    text = text.split(' ')
    for word in text:
        if word not in remove_word and word not in remove_trigram:
            new_text.append(word)
    return ' '.join(new_text)

In [ ]:
new_df['cleaned'] = new_df['phrases'].apply(remove_unnecessary)
print(new_df.shape)
new_df.head()

In [ ]:
filename = 'docket_texts/train/DT/basic_cleaned.pickle'

In [ ]:
#to save
with open(filename, 'wb') as handle: 
    pickle.dump(new_df, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [ ]:
new_df[['Original Docket Text', 'normalized', 'DT Topics', 'lemmed', 'phrases']].drop_duplicates().to_csv('Check if keywords working.csv', index = False)

In [ ]:
print('shape before dedupe: {}'.format(new_df.shape))
print('shape after dedupe: {}'.format(new_df[['Original Docket Text', 'normalized', 'DT Topics', 'lemmed', 'phrases']].drop_duplicates().shape))

In [ ]:
chris_file = pd.read_excel(r'temporary file - Sept 19.xlsx')
print(chris_file.shape)
chris_file.tail(1)

In [ ]:
another_df = new_df[['Original Docket Text', 'normalized', 'lemmed', 'phrases', 'DT Topics', 'NoAction']].drop_duplicates().merge(chris_file[['Original Docket Text', 'Action', 'If Action']], on = 'Original Docket Text', how = 'left')
print(another_df.shape)
another_df.tail(1)
#another_df.to_excel('tempfile.xlsx', index = False)

In [ ]:
another_df['Action revised'] = np.where(another_df['NoAction'] == 1, 'N', another_df['Action'])
another_df['If Action revised'] = np.where(another_df['NoAction'] == 1, '', another_df['If Action'])

another_df.head()

In [ ]:
#export to check
another_df.to_excel('tempfile.xlsx', index = False)
#applying methodology to the testing file is at another notebook